In [1]:
using DiffEqBase, OrdinaryDiffEq, ParameterizedFunctions, JLD2

#=
pf = @ode_def_bare MAIN begin
  # for rules
  v1 = k1*A*comp1
  # for dynamic
  dA = -v1/comp1
  dB = v1/comp1
end comp1=>1.1 k1=>0.01
=#

p_func = function(t, u, p, du)
    v1 = p[2]*u[1]*p[1]
    du[1] = -p[2]/p[1]
    du[2] = p[2]/p[1]
end
pf = ParameterizedFunction(p_func, [1.5,1.0])

#JLD2.@save "out.jld2" pf

u0 = [10., 0.]
tspan = (0., 100.)

prob = ODEProblem(pf,u0,tspan)

x=IOBuffer()
serialize(x, prob)

#open("ode_sys.bin", "w") do f
#    serialize(f, prob)
#end



In [8]:
workspace()

In [3]:
using Mongo, LibBSON
client = MongoClient("insilicobio.ru", 27017, "julia", "6dd580b4ed")
scenarios = MongoCollection(client, "testPlatform1", "scenarios")

insert(scenarios, ("id" => "new_ode", "odeDim" => 2,"ode_prob" => "$(x.data)"))

MongoCollection(scenarios)

In [5]:
new_ode = [m for m in find(scenarios, Dict("id"=>"new_ode"))]

1-element Array{LibBSON.BSONObject,1}:
 BSONObject({ "_id" : { "$oid" : "5a4226313be02425a80068b3" }, "odeDim" : 2, "id" : "new_ode", "ode_prob" : "UInt8[0x34, 0x10, 0x01, 0x0a, 0x4f, 0x44, 0x45, 0x50, 0x72, 0x6f, 0x62, 0x6c, 0x65, 0x6d, 0x1f, 0x01, 0x0a, 0x44, 0x69, 0x66, 0x66, 0x45, 0x71, 0x42, 0x61, 0x73, 0x65, 0x37, 0x07, 0x00, 0x00, 0x00, 0x10, 0x43, 0x1f, 0x8e, 0x37, 0x02, 0x00, 0x00, 0x00, 0x00, 0x0e, 0xc6, 0x00, 0x0e, 0x40, 0x10, 0x01, 0x15, 0x50, 0x61, 0x72, 0x61, 0x6d, 0x65, 0x74, 0x65, 0x72, 0x69, 0x7a, 0x65, 0x64, 0x46, 0x75, 0x6e, 0x63, 0x74, 0x69, 0x6f, 0x6e, 0x1f, 0x2c, 0x02, 0x00, 0x37, 0x03, 0x00, 0x00, 0x00, 0x40, 0x33, 0x13, 0x00, 0x00, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x01, 0x06, 0x23, 0x23, 0x39, 0x23, 0x31, 0x30, 0x22, 0x00, 0x00, 0x00, 0x00, 0x10, 0x4b, 0x1f, 0x8e, 0x37, 0x22, 0x00, 0x00, 0x00, 0x00, 0x22, 0x00, 0x00, 0x00, 0x00, 0x40, 0x3f, 0x3f, 0xa4, 0x01, 0x02, 0x23, 0x39, 0x15, 0x39, 0xc6, 0x23, 0x01, 0x00, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x1f, 0x37, 

In [7]:
ode_sys = eval(parse(new_ode[1]["ode_prob"]))

538-element Array{UInt8,1}:
 0x34
 0x10
 0x01
 0x0a
 0x4f
 0x44
 0x45
 0x50
 0x72
 0x6f
 0x62
 0x6c
 0x65
    ⋮
 0x59
 0x40
 0x41
 0x34
 0x2c
 0x21
 0x00
 0xc6
 0x34
 0x2c
 0x23
 0x00

In [8]:
io = IOBuffer(ode_sys)

IOBuffer(data=UInt8[...], readable=true, writable=false, seekable=true, append=false, size=538, maxsize=Inf, ptr=1, mark=-1)

In [ ]:
seekstart(io)
prob = deserialize(io)
#u0 = [10., 0.]
#tspan = (0., 100.)

#prob = ODEProblem(pf,u0,tspan)

sol = solve(prob, Tsit5(), saveat=1.)

In [108]:
using DiffEqBase, OrdinaryDiffEq, ParameterizedFunctions
#prob = deserialize(open("ode_sys.bin"))
seekstart(x)
prob = deserialize(x)
#u0 = [10., 0.]
#tspan = (0., 100.)

#prob = ODEProblem(pf,u0,tspan)

sol = solve(prob, Tsit5(), saveat=1.)

retcode: Success
Interpolation: 1st order linear
t: 101-element Array{Float64,1}:
   0.0
   1.0
   2.0
   3.0
   4.0
   5.0
   6.0
   7.0
   8.0
   9.0
  10.0
  11.0
  12.0
   ⋮  
  89.0
  90.0
  91.0
  92.0
  93.0
  94.0
  95.0
  96.0
  97.0
  98.0
  99.0
 100.0
u: 101-element Array{Array{Float64,1},1}:
 [10.0, 0.0]        
 [9.33333, 0.666667]
 [8.66667, 1.33333] 
 [8.0, 2.0]         
 [7.33333, 2.66667] 
 [6.66667, 3.33333] 
 [6.0, 4.0]         
 [5.33333, 4.66667] 
 [4.66667, 5.33333] 
 [4.0, 6.0]         
 [3.33333, 6.66667] 
 [2.66667, 7.33333] 
 [2.0, 8.0]         
 ⋮                  
 [-49.3333, 59.3333]
 [-50.0, 60.0]      
 [-50.6667, 60.6667]
 [-51.3333, 61.3333]
 [-52.0, 62.0]      
 [-52.6667, 62.6667]
 [-53.3333, 63.3333]
 [-54.0, 64.0]      
 [-54.6667, 64.6667]
 [-55.3333, 65.3333]
 [-56.0, 66.0]      
 [-56.6667, 66.6667]

In [104]:
typeof(x)

Base.AbstractIOBuffer{Array{UInt8,1}}

In [257]:
using DiffEqBase, OrdinaryDiffEq, ParameterizedFunctions

opts = Dict{Symbol,Bool}(
    :build_tgrad => false,
    :build_jac => false,
    :build_expjac => false,
    :build_invjac => false,
    :build_invW => false,
    :build_hes => false,
    :build_invhes => false,
    :build_dpfuncs => false)

ode1 = quote
    v1 = k1*A*comp1
    dA = -k1*A*comp1/comp1
    dB = k1*A*comp1/comp1
end

name = Symbol("MAIN")
params = :[:(comp1=>1.1), :(k1=>0.01)]

pf = @ode_def_bare(name1, $opts, $ode1, $(params))
#pf = ode_def_opts(name, opts, ode1, params)

u0 = [10., 0.]
tspan = (0., 1000.)

#prob = ODEProblem(pf,u0,tspan)


name=name1
opts=Dict(:build_dpfuncs=>false,:build_expjac=>false,:build_hes=>false,:build_jac=>false,:build_invhes=>false,:build_tgrad=>false,:build_invW=>false,:build_invjac=>false)
ex=$(Expr(:$, :opts))
params=(:($(Expr(:$, :ode1))), :($(Expr(:$, :params))))
expr=Expr[:(mutable struct name1 <: AbstractParameterizedFunction{true} # C:\Julia\v0.6\ParameterizedFunctions\src\maketype.jl, line 25:
        origex::Expr # C:\Julia\v0.6\ParameterizedFunctions\src\maketype.jl, line 26:
        funcs::Vector{Expr} # C:\Julia\v0.6\ParameterizedFunctions\src\maketype.jl, line 27:
        symfuncs::Vector{SymEngine.Basic} # C:\Julia\v0.6\ParameterizedFunctions\src\maketype.jl, line 28:
        pfuncs::Vector{Expr} # C:\Julia\v0.6\ParameterizedFunctions\src\maketype.jl, line 29:
        d_pfuncs::Vector{Expr} # C:\Julia\v0.6\ParameterizedFunctions\src\maketype.jl, line 30:
        syms::Vector{Symbol} # C:\Julia\v0.6\ParameterizedFunctions\src\maketype.jl, line 31:
        symtgrad::Vector{SymEngin

LoadError: [91mMethodError: Cannot `convert` an object of type Dict{Symbol,Bool} to an object of type Expr
This may have arisen from a call to the constructor Expr(...),
since type constructors fall back to convert methods.[39m

In [48]:
update(scenarios, ("_id" => id), inc("age" => 1))

In [85]:
command_simple(client,
               "testPlatform1",
               Dict(
                  "distinct" => "scenarios",
                  "indexes" => [
                      Dict(
                          "key" => Dict("name" => 1),
                          "name" => "cats_name",
                          "unique" => 1)
                      ]
                  ))

{ "_id" : { "$oid" : "5a3b8d3a35a8555a6a1b5fc0" }, "id" : "default", "odeDim" : 2, "ode" : [ "# for rules", "v1 = k1*A*comp1", "# for dynamic", "dA = -v1/comp1", "dB = v1/comp1" ], "events" : [ { "condition" : [ "isequal(t,0)" ], "affect!" : [ "# for dynamic", "integrator.u[1] = p[3]", "integrator.u[2] = p[4]", "# for static", "integrator.f.comp1 = p[1]", "integrator.f.k1 = p[2]" ] } ] }
{ "_id" : { "$oid" : "5a40efe612f03800a854666f" }, "id" : "default", "odeDim" : 2, "ode" : [ "# for rules", "v1 = k1*A*comp1", "# for dynamic", "dA = -v1/comp1", "dB = v1/comp1" ], "events" : [ { "condition" : [ "isequal(t,0)" ], "affect!" : [ "# for dynamic", "integrator.u[1] = p[3]", "integrator.u[2] = p[4]", "# for static", "integrator.f.comp1 = p[1]", "integrator.f.k1 = p[2]" ] } ] }


LoadError: [91mUndefVarError: new_ode not defined[39m

In [26]:
using DiffEqBase, OrdinaryDiffEq, ParameterizedFunctions, JLD2

JLD2.@load "out.jld2" pf

u0 = [10., 0.]
tspan = (0., 1000.)

prob = ODEProblem(pf,u0,tspan)
sol = solve(prob, Tsit5(), saveat=1.)

retcode: Success
Interpolation: 1st order linear
t: 1001-element Array{Float64,1}:
    0.0
    1.0
    2.0
    3.0
    4.0
    5.0
    6.0
    7.0
    8.0
    9.0
   10.0
   11.0
   12.0
    ⋮  
  989.0
  990.0
  991.0
  992.0
  993.0
  994.0
  995.0
  996.0
  997.0
  998.0
  999.0
 1000.0
u: 1001-element Array{Array{Float64,1},1}:
 [10.0, 0.0]        
 [9.33333, 0.666667]
 [8.66667, 1.33333] 
 [8.0, 2.0]         
 [7.33333, 2.66667] 
 [6.66667, 3.33333] 
 [6.0, 4.0]         
 [5.33333, 4.66667] 
 [4.66667, 5.33333] 
 [4.0, 6.0]         
 [3.33333, 6.66667] 
 [2.66667, 7.33333] 
 [2.0, 8.0]         
 ⋮                  
 [-649.333, 659.333]
 [-650.0, 660.0]    
 [-650.667, 660.667]
 [-651.333, 661.333]
 [-652.0, 662.0]    
 [-652.667, 662.667]
 [-653.333, 663.333]
 [-654.0, 664.0]    
 [-654.667, 664.667]
 [-655.333, 665.333]
 [-656.0, 666.0]    
 [-656.667, 666.667]